In [ ]:
pip install pandas-profiling

In [ ]:
!pip install -U dataprep

In [ ]:
import pandas as pd
from pandas import Series
import pandas_profiling
from pandas_profiling import ProfileReport
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
#import dataprep.eda
from datetime import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
class Model(): # класс для оценки метрик модели
    def __init__(self, model, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        self.y_pred = model.predict(X_test)
        self.probs = model.predict_proba(X_test)[:,1]
    
    def roc_curve(self):
        fpr, tpr, threshold = roc_curve(self.y_test, self.probs)
        roc_auc = roc_auc_score(self.y_test, self.probs)

        plt.figure()
        plt.plot([0, 1], label='Baseline', linestyle='--')
        plt.plot(fpr, tpr, label = 'Regression')
        plt.title('Logistic Regression ROC AUC = %0.05f' % roc_auc)
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.legend(loc = 'lower right')
        plt.show()
    
    def confusion_matrix(self):
        tn, fp, fn, tp = confusion_matrix(self.y_test, self.y_pred).ravel()
        cf_matrix = np.array([[tp,fp],[fn,tn]])
        group_names = ['TP','FP','FN','TN']
        group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
        labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_names,group_counts)]
        labels = np.asarray(labels).reshape(2,2)
        plt.figure()
        sns.heatmap(cf_matrix, annot=labels, annot_kws={"size": 20}, fmt='', cmap= 'Pastel1', cbar = False, \
                 xticklabels = ['Дефолт','Не дефолт'], yticklabels= ['Дефолт','Не дефолт'])
        plt.title('Матрица ошибок')
        plt.show()
        
    def get_metrics(self):
        result = pd.Series({
            'Accuracy' : accuracy_score(self.y_test, self.y_pred),
            'Precision' : precision_score(self.y_test, self.y_pred),
            'Recall' : recall_score(self.y_test, self.y_pred),
            'F1' : f1_score(self.y_test, self.y_pred),
            'ROC_AUC': roc_auc_score(self.y_test, self.probs) 
        })
        return result

In [ ]:
RANDOM_SEED = 42
!pip freeze > requirements.txt

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
train = pd.read_csv(DATA_DIR+'/train.csv')
test = pd.read_csv(DATA_DIR+'test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

Посмотрим на данные трейна и теста.

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# для корректной обработки признаков объединяем трейн и тест в один датасет
train['Train'] = 1 # помечаем где у нас трейн
test['Train'] = 0 # помечаем где у нас тест

data = train.append(test, sort=False).reset_index(drop=True) # объединяем

In [ ]:
# Информация о пропусках во всём датасете.
data.isna().sum()

В итоге видим, что в тестовой выборке отсутствует колонка default, соответственно большое кол-во пропусков в общем датасете.

In [ ]:
data

## Описание датасета
* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заемщика
* age - возраст заемщика
* car - наличие автомобиля
* car_type - наличие автомобиля иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - наличие хорошей работы
* bki_request_cnt - количество запросов в БКИ
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заемщика
* foreign_passport - наличие загранпаспорта
* sna - связь заемщика с клиентами банка
* first_time - давность наличия информации о заемщике
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона
* app_date - дата подачи заявки
* default - дефолт по кредиту

In [ ]:
# Кол-во дубликатов. (На всякий случай)
data.duplicated().sum()

In [ ]:
# Распределение целевой переменной
train['default'].value_counts().plot.bar()

Увы, но неравномерное.

## Далее рассмотрим подробнее наши признаки:

In [ ]:
ProfileReport(data)

In [ ]:
# Зададим списки числовых, категориальных и бинарных признаков
num_cols = ['age','score_bki', 'bki_request_cnt','decline_app_cnt', 'income']
cat_cols = ['home_address', 'work_address', 'sna', 'first_time', 'region_rating','education']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [ ]:
dataprep.eda.plot(data, 'age')
# Видим, что самый молодой заёмщик 21 год, а самый старый 72. Средний возраст 39 лет.

In [ ]:
dataprep.eda.plot(data, 'score_bki')
# Видимо признак был ранее прологарифмирован. Трогать его не будем.

In [ ]:
dataprep.eda.plot(data, 'bki_request_cnt')

In [ ]:
count_bki = data.groupby('age')['bki_request_cnt'].count().to_dict()
count_bki
# Количество обращений больше в 26-32 года.

In [ ]:
dataprep.eda.plot(data, 'decline_app_cnt')

In [ ]:
dataprep.eda.plot(data, 'income')

In [ ]:
dataprep.eda.plot(train, 'income', 'default')
# Видим, что дефолтные заёмщики с невысоким доходом.

In [ ]:
# Теперь прологарифмируем признаки.
data['age'] = np.log(data['age'] + 1)
data['income'] = np.log(data['income'] + 1)
data['decline_app_cnt'] = np.log(data['decline_app_cnt'] + 1)
data['bki_request_cnt'] = np.log(data['bki_request_cnt'] + 1)

Для категориальных признаков будем делать get_dummies.

In [ ]:
dataprep.eda.plot(data, 'education')
# Мы видим, что со школьным образованием больше всего, почти 53%. 
# А людей с академическим образованием меньше всех, почти 0,3%. 
# Также видим тенденцию - чем лучше образование, тем меньше берут кредиты.

In [ ]:
# Пропусков не так много, поэтому заполним школьным образованием.
data['education'] = data['education'].fillna('SCH')

In [ ]:
dataprep.eda.plot(data, 'region_rating')

In [ ]:
dataprep.eda.plot(data, 'region_rating', 'default')
# Чем ниже рейтинг региона, тем вероятнее невыплата кредита.

In [ ]:
dataprep.eda.plot(data, 'first_time')

In [ ]:
dataprep.eda.plot(data, 'sna')

In [ ]:
dataprep.eda.plot(data, 'home_address')

In [ ]:
dataprep.eda.plot(data, 'work_address')

In [ ]:
dataprep.eda.plot(data, 'sex')
# Вывод простой - женщины берут кредиты чаще, чем мужчины, правда не намного: 56% против 44%.

In [ ]:
dataprep.eda.plot(data, 'car')
# Треть имеет автомобиль

In [ ]:
dataprep.eda.plot(data, 'car_type')
# Три четверти заёмщиков ездит на отечественном автомобиле.

In [ ]:
dataprep.eda.plot(data, 'good_work')

In [ ]:
dataprep.eda.plot(data, 'foreign_passport')
# У 15% есть загранпаспорт.

In [ ]:
# Закодируем бинарные переменные.
label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])

In [ ]:
# Переведём колонку в цифровой формат.
edu_dict = {'SCH': 1,'UGR': 2,'GRD': 3,'PGR': 4,'ACD': 5}
data['education'] = data['education'].map(edu_dict)

In [ ]:
# Преобразуем формат признака app_date
data.app_date = pd.to_datetime(data.app_date, format='%d%b%Y')

In [ ]:
# Начало и конец периода нашего датасета
start = data.app_date.min()
end = data.app_date.max()
print(start, end)

In [ ]:
# Новые признаки

max_income = data.groupby('age')['income'].max().to_dict()
data['sign_1'] = data['age'].map(max_income)

mean_income_region = data.groupby('region_rating')['income'].mean().to_dict()
data['sign_2'] = data['region_rating'].map(mean_income_region)

mean_income_age = data.groupby('age')['income'].mean().to_dict()
data['sign_3'] = data['age'].map(mean_income_age)

mean_bki = data.groupby('age')['bki_request_cnt'].mean().to_dict()
data['sign_4'] = data['age'].map(mean_bki)

data['sign_5'] = data.region_rating * data.car
data['sign_6'] = data.home_address + data.work_address
data['sign_7'] = data.age / data.score_bki
data['sign_8'] = (data.home_address + data.work_address) * data.good_work 
data['sign_9'] = (data.home_address + data.work_address) * data.sna 
data['sign_10'] = data.decline_app_cnt * data.sna
data['sign_11'] = data.income - data.age.map(mean_income_age)
data['sign_12'] = data.income/data.age
data['sign_13'] = data.education/data.age

new_cols = ['sign_1', 'sign_2', 'sign_3', 'sign_4', 'sign_5', 'sign_6', 'sign_7', 'sign_8', 'sign_9', 'sign_10',
           'sign_11', 'sign_12', 'sign_13']

In [ ]:
# Удалим из датасета признак app_date, он нам больше не понадобится.
data.drop(['app_date'],  axis = 1, inplace = True)

In [ ]:
# Нормализуем данные.
num_cols = ['age', 'decline_app_cnt', 'bki_request_cnt', 'income']
scaler = StandardScaler()
for col in num_cols:
    data[col] = scaler.fit_transform(data[[col]])
for col in new_cols:
    data[col] = scaler.fit_transform(data[[col]]) 

In [ ]:
# Преобразуем категориальные признаки.
data=pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)

## Перейдём к модели.

In [ ]:
# Разбиваем датасет на тренировочный и тестовый.
train_data = data.query('Train == 1').drop(['Train', 'client_id'], axis=1)
test_data = data.query('Train == 0').drop(['Train', 'client_id'], axis=1)

y = train_data.default.values
X = train_data.drop(['default'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### Подбираем параметры для модели LogisticRegression. Код закомментирован, т.к. выполняется за длительное время. 

In [ ]:
#model = LogisticRegression(random_state=RANDOM_SEED)

#iter_ = 100
#epsilon_stop = 1e-3

#param_grid = [
#    {'penalty': ['l1'], 
#     'solver': ['liblinear', 'lbfgs'], 
#     'class_weight':['none', 'balanced'], 
#     'multi_class': ['auto','ovr'], 
#     'max_iter':[iter_],
#     'tol':[epsilon_stop]},
#    {'penalty': ['l2'], 
#     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
#     'class_weight':['none', 'balanced'], 
#     'multi_class': ['auto','ovr'], 
#     'max_iter':[iter_],
#     'tol':[epsilon_stop]},
#    {'penalty': ['none'], 
#     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
#     'class_weight':['none', 'balanced'], 
#     'multi_class': ['auto','ovr'], 
#     'max_iter':[iter_],
#     'tol':[epsilon_stop]},
#]
#gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
#gridsearch.fit(X_train, y_train)
#model = gridsearch.best_estimator_
# Параметры
#best_parameters = model.get_params()
#for param_name in sorted(best_parameters.keys()):
#        print('\t%s: %r' % (param_name, best_parameters[param_name]))

In [ ]:
log_model = LogisticRegression(random_state=42, 
                           C=1.0, 
                           class_weight= 'balanced', 
                           dual= False, 
                           fit_intercept= True, 
                           intercept_scaling= 1, 
                           l1_ratio= None,
                           max_iter = 100,
                           multi_class= 'auto', 
                           n_jobs= None,
                           penalty = 'l2',
                           solver = 'lbfgs',
                           tol = 0.001,
                           verbose= 0, 
                           warm_start= False)

log_model.fit(X_train, y_train)

y_pred_prob = log_model.predict_proba(X_test)[:,1]
y_pred = log_model.predict(X_test)

fpr, tpr, threshold = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.5f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

print('accuracy_score:', accuracy_score(y_test, y_pred))
print('precision_score:', precision_score(y_test, y_pred))
print('recall_score:', recall_score(y_test, y_pred))
print('f1_score:', f1_score(y_test, y_pred))
print('roc_auc:', roc_auc_score(y_test, y_pred_proba))
model_1 = Model(log_model, X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=9, class_weight= 'balanced', random_state=42)
clf.fit(X_train, y_train)

y_pred_proba = clf.predict_proba(X_test)[:, 1]

y_pred = clf.predict(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.5f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

print('accuracy_score:', accuracy_score(y_test, y_pred))
print('precision_score:', precision_score(y_test, y_pred))
print('recall_score:', recall_score(y_test, y_pred))
print('f1_score:', f1_score(y_test, y_pred))
print('roc_auc:', roc_auc_score(y_test, y_pred_proba))
model_2 = Model(clf, X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth = 6, class_weight= 'balanced')
dtc.fit(X_train, y_train)

y_pred_proba = dtc.predict_proba(X_test)[:, 1]

y_pred = dtc.predict(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.5f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

print('accuracy_score:', accuracy_score(y_test, y_pred))
print('precision_score:', precision_score(y_test, y_pred))
print('recall_score:', recall_score(y_test, y_pred))
print('f1_score:', f1_score(y_test, y_pred))
print('roc_auc:', roc_auc_score(y_test, y_pred_proba))
model_3 = Model(dtc, X_test, y_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=500,learning_rate=0.1,max_depth=1)
gb.fit(X_train, y_train)

y_pred_proba = gb.predict_proba(X_test)[:, 1]

y_pred = gb.predict(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.5f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

print('accuracy_score:', accuracy_score(y_test, y_pred))
print('precision_score:', precision_score(y_test, y_pred))
print('recall_score:', recall_score(y_test, y_pred))
print('f1_score:', f1_score(y_test, y_pred))
print('roc_auc:', roc_auc_score(y_test, y_pred_proba))
model_4 = Model(gb, X_test, y_test)

In [ ]:
# Посмотрим на показатели моделей.
metrics = pd.concat([model_1.get_metrics(), model_2.get_metrics(),
                     model_3.get_metrics(), model_4.get_metrics()],axis = 1)
metrics.columns = ['model_1', 'model_2', 'model_3', 'model_4']
metrics

In [ ]:
model_1.confusion_matrix()
model_2.confusion_matrix()
model_3.confusion_matrix()
model_4.confusion_matrix()

## Выводы:
* Использовал (протестировал) несколько моделей.
* Подбирал лучшие параметры для моделей, которые улучшали её. Но только для логистической регрессии, для деревьев и градиентного бустинга не успел.
* С выбросами боролся логарифмированием.
* бОльшая часть новых признаков ухудшала показатели модели. Также не успел провести работу с признаками, которые могли бы ухудшить модель.
* Погуглил и нашёл информацию, что при логистической регрессии хорошо идут математические операции с признаками (при создании новых), что собстсвенно и использовал.
* Хорошие данные важнее, чем хороший алгоритм!
* В дальнейшем буду продолжать вести работу с данными и улучшать результат.

In [ ]:
X_test = test_data.drop(['default'], axis=1)
y_pred_prob = log_model.predict_proba(X_test)[:,1]

submit = pd.DataFrame(test.client_id)
submit['default']=y_pred_prob
submit.to_csv('submission.csv', index=False)